In [1]:
from pgsql_search.loader import HuggingFaceDatasets

ds = HuggingFaceDatasets("UCSC-VLAA/Recap-COCO-30K")
# ds.dataset = ds.dataset.shuffle()
# ds = ds.select(list(range(100)))
ds.save_images("../data/images")
ds = ds.select_columns(["image_filepath", "caption"])
ds.dataset


2024-12-20 14:19:02.523 | INFO     | pgsql_search.loader:__init__:15 - Loading dataset: UCSC-VLAA/Recap-COCO-30K
2024-12-20 14:19:02.525 | INFO     | pgsql_search.loader:save_images:44 - Saving images to folder: ../data/images


Saving images (num_proc=8):   0%|          | 0/30504 [00:00<?, ? examples/s]

Dataset({
    features: ['image_filepath', 'caption'],
    num_rows: 30504
})

In [2]:
df = ds.dataset.to_pandas()
df

,image_filepath,caption
0,/home/dnth/Desktop/pgsql-search/data/images/53...,Several horses with riders crossing a body of ...
1,/home/dnth/Desktop/pgsql-search/data/images/32...,A woman and a child sit on the floor and inter...
2,/home/dnth/Desktop/pgsql-search/data/images/21...,There is an open face sandwich and a fork on t...
3,/home/dnth/Desktop/pgsql-search/data/images/19...,a pillow a bed a table and two books
4,/home/dnth/Desktop/pgsql-search/data/images/42...,Three elephants in a field next to each other.
...,...,...
30499,/home/dnth/Desktop/pgsql-search/data/images/28...,The light shows safe for animal crossing on th...
30500,/home/dnth/Desktop/pgsql-search/data/images/40...,A crowd of people watching an indoor tennis ma...
30501,/home/dnth/Desktop/pgsql-search/data/images/52...,"A street corner has a sign, and across the str..."
30502,/home/dnth/Desktop/pgsql-search/data/images/13...,motorcycles parked on the side of a road near ...


In [3]:
from pgsql_search.database import PostgreSQLDatabase, ColumnType

PostgreSQLDatabase.create_database("my_database")

2024-12-20 14:19:29.555 | INFO     | pgsql_search.database:create_database:361 - Database 'my_database' already exists


In [4]:
with PostgreSQLDatabase("my_database") as db:
    # First, create the table with just an ID column
    db.initialize_table("image_metadata")
    db.add_column("image_filepath", ColumnType.TEXT, nullable=False)
    db.add_column("caption", ColumnType.TEXT, nullable=True)
    db.insert_dataframe(df)

2024-12-20 14:19:29.649 | INFO     | pgsql_search.database:connect:116 - Connected to database
2024-12-20 14:19:29.654 | INFO     | pgsql_search.database:setup_pgvector_extension:133 - pgvector extension initialized
2024-12-20 14:19:29.660 | INFO     | pgsql_search.database:initialize_table:155 - Initialized table 'image_metadata' with ID column 'id'
2024-12-20 14:19:29.662 | INFO     | pgsql_search.database:add_columns:204 - Added 1 new columns [image_filepath] to image_metadata
2024-12-20 14:19:29.663 | INFO     | pgsql_search.database:add_columns:204 - Added 1 new columns [caption] to image_metadata
2024-12-20 14:19:29.668 | INFO     | pgsql_search.database:insert_dataframe:241 - Table columns: {'id': 'integer', 'image_filepath': 'text', 'caption': 'text'}
2024-12-20 14:19:29.669 | INFO     | pgsql_search.database:insert_dataframe:242 - DataFrame columns: Index(['image_filepath', 'caption'], dtype='object')
2024-12-20 14:19:29.670 | INFO     | pgsql_search.database:insert_dataframe: